# REDES NEURAIS ARTIFICIAIS

[Aula 3 de Deep Learning](http://bit.ly/dn-unb03) da Engenharia de Software da UnB

# Parte 1 - Pré-processamento dos Dados

In [5]:
!pip install -r requirements.txt

     |████████████████████████████████| 17.3 MB 3.8 MB/s eta 0:00:01
     |████████████████████████████████| 10.1 MB 34.9 MB/s eta 0:00:01
     |████████████████████████████████| 13.0 MB 13.1 MB/s eta 0:00:01
     |████████████████████████████████| 320.4 MB 19 kB/s /s eta 0:00:01█████████▍                 | 143.7 MB 78.9 MB/s eta 0:00:03��███████████████▎ | 303.4 MB 125.0 MB/s eta 0:00:01
     |████████████████████████████████| 459 kB 50.4 MB/s eta 0:00:01
     |████████████████████████████████| 10.6 MB 48.7 MB/s eta 0:00:01�█████████████▉  | 9.9 MB 48.7 MB/s eta 0:00:01
     |████████████████████████████████| 2.9 MB 28.3 MB/s eta 0:00:01
     |████████████████████████████████| 65 kB 6.8 MB/s  eta 0:00:01
     |████████████████████████████████| 1.0 MB 44.0 MB/s eta 0:00:01
     |████████████████████████████████| 57 kB 6.5 MB/s  eta 0:00:01
     |████████████████████████████████| 781 kB 53.8 MB/s eta 0:00:01
     |████████████████████████████████| 136 kB 51.4 MB/s eta 0:00:01
     |████

### Importar as libs

In [6]:
import numpy as np
import pandas as pd
import tensorflow as tf

In [7]:
tf.__version__

'2.3.1'

### Importar o dataset

In [8]:
dataset = pd.read_csv('Churn_Modelling.csv')
dataset.head()

,RowNumber,CustomerId,Surname,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,1,15634602,Hargrave,619,France,Female,42,2,0.00,1,1,1,101348.88,1
1,2,15647311,Hill,608,Spain,Female,41,1,83807.86,1,0,1,112542.58,0
2,3,15619304,Onio,502,France,Female,42,8,159660.80,3,1,0,113931.57,1
3,4,15701354,Boni,699,France,Female,39,1,0.00,2,0,0,93826.63,0
4,5,15737888,Mitchell,850,Spain,Female,43,2,125510.82,1,1,1,79084.10,0


In [9]:
X = dataset.iloc[:, 3:13].values
y = dataset.iloc[:, 13].values
print("X >>\n",X)
print("y >>\n",y)

X >>
 [[619 'France' 'Female' ... 1 1 101348.88]
 [608 'Spain' 'Female' ... 0 1 112542.58]
 [502 'France' 'Female' ... 1 0 113931.57]
 ...
 [709 'France' 'Female' ... 0 1 42085.58]
 [772 'Germany' 'Male' ... 1 0 92888.52]
 [792 'France' 'Female' ... 1 0 38190.78]]
y >>
 [1 0 1 ... 1 1 0]


### Transformando os dados categóricos


In [10]:
# Label Encoding the "Gender" column
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
X[:, 2] = le.fit_transform(X[:, 2])
print(X)
# One Hot Encoding the "Geography" column
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder
ct = ColumnTransformer(transformers=[('encoder', OneHotEncoder(), [1])], remainder='passthrough')
X = np.array(ct.fit_transform(X))
print(X)

[[619 'France' 0 ... 1 1 101348.88]
 [608 'Spain' 0 ... 0 1 112542.58]
 [502 'France' 0 ... 1 0 113931.57]
 ...
 [709 'France' 0 ... 0 1 42085.58]
 [772 'Germany' 1 ... 1 0 92888.52]
 [792 'France' 0 ... 1 0 38190.78]]
[[1.0 0.0 0.0 ... 1 1 101348.88]
 [0.0 0.0 1.0 ... 0 1 112542.58]
 [1.0 0.0 0.0 ... 1 0 113931.57]
 ...
 [1.0 0.0 0.0 ... 0 1 42085.58]
 [0.0 1.0 0.0 ... 1 0 92888.52]
 [1.0 0.0 0.0 ... 1 0 38190.78]]


# Dividindo o dataset em conjunto de treinamento e conjunto de teste

In [11]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 0)

# Feature Scaling

In [12]:
from sklearn.preprocessing import StandardScaler
sc = StandardScaler()
X_train = sc.fit_transform(X_train)
X_test = sc.transform(X_test)

In [13]:
np.shape(X_train)

(8000, 12)

---
# Parte 2 -Vamos construir uma ANN!


### Initializing the ANN

In [14]:
ann = tf.keras.models.Sequential()

### Adding the input layer and the first hidden layer

In [15]:
ann.add(tf.keras.layers.Dense(units=6, activation='softsign'))

### Adding the second hidden layer

In [16]:
ann.add(tf.keras.layers.Dense(units=6, activation='relu'))

### Adding the output layer

In [17]:
ann.add(tf.keras.layers.Dense(units=1, activation='sigmoid'))

## Part 3 - Training the ANN

### Compiling the ANN

In [18]:
ann.compile(optimizer = 'adam', loss = 'binary_crossentropy', metrics = ['accuracy'])

### Training the ANN on the Training set

In [ ]:
ann.fit(X_train, y_train, batch_size = 32, epochs = 100)

Epoch 1/100
250/250 [==============================] - 1s 2ms/step - loss: 0.6014 - accuracy: 0.7395
Epoch 2/100
250/250 [==============================] - 0s 2ms/step - loss: 0.4779 - accuracy: 0.7960
Epoch 3/100
250/250 [==============================] - 0s 2ms/step - loss: 0.4465 - accuracy: 0.7959
Epoch 4/100
250/250 [==============================] - 0s 2ms/step - loss: 0.4320 - accuracy: 0.7968
Epoch 5/100
250/250 [==============================] - 0s 2ms/step - loss: 0.4257 - accuracy: 0.8077
Epoch 6/100
250/250 [==============================] - 1s 2ms/step - loss: 0.4206 - accuracy: 0.8142
Epoch 7/100
250/250 [==============================] - 0s 2ms/step - loss: 0.4155 - accuracy: 0.8184
Epoch 8/100
250/250 [==============================] - 1s 2ms/step - loss: 0.4107 - accuracy: 0.8207
Epoch 9/100
250/250 [==============================] - 1s 2ms/step - loss: 0.4062 - accuracy: 0.8253
Epoch 10/100
250/250 [==============================] - 1s 2ms/step - loss: 0.4013 - accura

# Parte 3 - Fazendo predições e avaliando o modelo

## Prevendo os resultados com o conjunto de testes

In [ ]:
y_pred = ann.predict(X_test)
y_pred

In [ ]:
y_pred = (y_pred > 0.5)
y_pred

# Criando uma Confusion Matrix

In [ ]:
from sklearn.metrics import confusion_matrix
cm = confusion_matrix(y_test, y_pred)
print(cm)

# FIM

> Professor Diego Dorgam  
> [@diegodorgam](https://twitter.com/diegodorgam)